In [1]:
%run benchmarking_functions-LungHisto.ipynb

In [2]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# ✅ Assume dataset already loaded before this script:
# (x_train, y_train), (x_test, y_test), input_shape = load_dataset(root='lung_image_sets')

results = []

if __name__ == "__main__":
    datasets = ['Histo/lung_colon_image_set/lung_image_sets']  # your custom dataset name
    encodings = ['poisson_only', 'cnn+poisson']
    architectures = ['1SADP', '2SADP']
    timesteps = [25, 100]
    n_internal_epochs = 50  

    for dname in datasets:
        print("\n" + "#" * 80)
        print(f"Running full SNN pipeline for {dname.upper()}")
        print("#" * 80)

        # 🟡 Dataset already loaded, no load_dataset() call

        for arch in architectures:
            print("\n" + "=" * 80)
            print(f"Architecture: {arch}")
            print("=" * 80)

            for T_val in timesteps:
                print("\n" + "~" * 80)
                print(f"Time Steps: {T_val}")
                print("~" * 80)

                for encoding in encodings:
                    print("\n" + "-" * 60)
                    print(f"Encoding type: {encoding}")
                    print("-" * 60)

                    # ---- Encoder / Feature extraction ----
                    run_experiment(
                        dataset_name=dname,
                        feature_dim=256,
                        encoder_epochs=50,
                        batch_size_encoder=128,
                        pretrain_encoder=True,
                        Nhid=256,
                        theta_h_base=0.5,
                        seed=42,
                        architecture=arch,
                        T=T_val,
                        encoding_type=encoding
                    )

                    # ---- Train SNN ----
                    print(f"\nTraining SNN on {dname.upper()} features ({encoding}, {arch}, T={T_val})...")
                    t_start = time.time()

                  
                    acc, n_epochs_run, epoch_accs = train_snn(
                    n_epochs=n_internal_epochs,
                    batch_size=128
                    )
                    
                    final_train_acc = epoch_accs[-1] if len(epoch_accs) > 0 else None
                    t_end = time.time()
                    total_time = t_end - t_start
                    avg_time_per_epoch = total_time / n_epochs_run

                    # ---- Evaluate SNN ----
                    print(f"\nEvaluating SNN on {dname.upper()} ({encoding}, {arch}, T={T_val})...")
                    acc_eval = evaluate_snn(batch_size=256)

                    # ---- Forward pass for metrics ----
                    sh, so = forward_lif_features(test_feats_norm, teacher_force=False)
                    preds = np.argmax(so.sum(axis=1), axis=1)
                    y_true = y_test

                    prec = precision_score(y_true, preds, average='macro', zero_division=0)
                    rec = recall_score(y_true, preds, average='macro', zero_division=0)
                    f1 = f1_score(y_true, preds, average='macro', zero_division=0)

                    results.append({
                        "Dataset": dname.upper(),
                        "Architecture": arch,
                        "Timestep": T_val,
                        "Encoding": encoding,
                        "Final Train Accuracy": acc,
                        "Eval Accuracy": acc_eval,
                        "Precision": prec,
                        "Recall": rec,
                        "F1-score": f1,
                        "Total Train Time (s)": total_time,
                        "Internal Epochs Run": n_epochs_run,
                        "Avg Time per Internal Epoch (s)": avg_time_per_epoch,
                        "Epoch Accuracies": epoch_accs
                    })

    # -----------------------------------------------------
    # ✅ Save Results
    # -----------------------------------------------------
    df_results = pd.DataFrame(results)

    print("\n" + "#" * 80)
    print("SNN Performance Summary")
    print("#" * 80)
    print(df_results)



################################################################################
Running full SNN pipeline for HISTO/LUNG_COLON_IMAGE_SET/LUNG_IMAGE_SETS
################################################################################

Architecture: 1SADP

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Time Steps: 25
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

------------------------------------------------------------
Encoding type: poisson_only
------------------------------------------------------------

Running experiment | 1SADP | T=25 | Encoding: poisson_only
Loading dataset: Histo/lung_colon_image_set/lung_image_sets
Found 12000 images belonging to 3 classes.
Found 3000 images belonging to 3 classes.
✅ Loaded LUNG_IMAGE_SETS dataset successfully
Input shape: (28, 28, 3)
Train: (12000, 28, 28, 3), Test: (3000, 28, 28, 3)
Classes found: 3
Feature dim (Nin): 2352, Nhid: 256, Nout: 3
W1: (2352, 256), W2: (256,

SNN Epoch 1/50: 100%|██████████████████████████████████████████████████████████████████| 93/93 [02:03<00:00,  1.33s/it]


Epoch 1/50 | Train acc = 0.5172 | avg κ = -0.001128 | Time = 123.66s


SNN Epoch 2/50: 100%|██████████████████████████████████████████████████████████████████| 93/93 [02:03<00:00,  1.33s/it]


Epoch 2/50 | Train acc = 0.6347 | avg κ = -0.001363 | Time = 123.81s


...

Epoch 49/50 | Train acc = 0.9854 | avg κ = 0.004838 | Time = 6.09s


SNN Epoch 50/50: 100%|█████████████████████████████████████████████████████████████████| 93/93 [00:06<00:00, 15.25it/s]


Epoch 50/50 | Train acc = 0.9854 | avg κ = 0.004781 | Time = 6.10s

Evaluating SNN on HISTO/LUNG_COLON_IMAGE_SET/LUNG_IMAGE_SETS (cnn+poisson, 2SADP, T=100)...


SNN Testing: 100%|█████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 12.79it/s]


SNN Test Acc = 0.9850

################################################################################
SNN Performance Summary
################################################################################
                                      Dataset Architecture  Timestep  \
0  HISTO/LUNG_COLON_IMAGE_SET/LUNG_IMAGE_SETS        1SADP        25   
1  HISTO/LUNG_COLON_IMAGE_SET/LUNG_IMAGE_SETS        1SADP        25   
2  HISTO/LUNG_COLON_IMAGE_SET/LUNG_IMAGE_SETS        1SADP       100   
3  HISTO/LUNG_COLON_IMAGE_SET/LUNG_IMAGE_SETS        1SADP       100   
4  HISTO/LUNG_COLON_IMAGE_SET/LUNG_IMAGE_SETS        2SADP        25   
5  HISTO/LUNG_COLON_IMAGE_SET/LUNG_IMAGE_SETS        2SADP        25   
6  HISTO/LUNG_COLON_IMAGE_SET/LUNG_IMAGE_SETS        2SADP       100   
7  HISTO/LUNG_COLON_IMAGE_SET/LUNG_IMAGE_SETS        2SADP       100   

       Encoding Final Train Accuracy  Eval Accuracy  Precision    Recall  \
0  poisson_only                 None       0.730000   0.735502  0.

In [3]:
poisson_df = df_results[df_results["Encoding"].str.contains("poisson_only", case=False, na=False)]
poisson_cnn_df = df_results[df_results["Encoding"].str.contains("cnn\\+poisson", case=False, na=False)]

if not poisson_cnn_df.empty:
    excel_filename = "LungHisto_results.xlsx"
    with pd.ExcelWriter(excel_filename, engine="openpyxl") as writer:
        poisson_df.to_excel(writer, sheet_name="Poisson_Only", index=False)
        poisson_cnn_df.to_excel(writer, sheet_name="CNN+Poisson", index=False)
    print(f"\n Results saved to '{excel_filename}' (two sheets: Poisson_Only & CNN+Poisson)")
else:
    csv_filename = "lunghisto-sadp_results_summary_poisson_only.csv"
    poisson_df.to_csv(csv_filename, index=False)
    print(f"\n Only Poisson results detected. Results saved to '{csv_filename}'")



 Results saved to 'LungHisto_results.xlsx' (two sheets: Poisson_Only & CNN+Poisson)
